In [ ]:
# from transformers import LayoutLMv3ForTokenClassification, LayoutLMv3Processor
# from PIL import Image, ImageDraw, ImageFont
# import torch

# model = LayoutLMv3ForTokenClassification.from_pretrained("test_trainer/checkpoint-2000")
# processor = LayoutLMv3Processor.from_pretrained("microsoft/layoutlmv3-base", apply_ocr=True)

# def unnormalize_box(bbox, width, height):
#     return [
#         width * (bbox[0] / 1000),
#         height * (bbox[1] / 1000),
#         width * (bbox[2] / 1000),
#         height * (bbox[3] / 1000),
#     ]


# label2id = {
#     'O': 0,
#     'B-title': 1,
#     'I-title': 2,
#     'B-date': 3,
#     'I-date': 4,
#     'B-ieee': 5,
#     'I-ieee': 6,
#     'B-total': 7,
#     'I-total': 8,
#     'B-totalValue': 9,
#     'I-totalValue': 10
# }
# id2label = {id: label for label, id in label2id.items()}

# # draw results onto the image
# def draw_boxes(image, boxes, predictions):
#     width, height = image.size
#     normalizes_boxes = [unnormalize_box(box, 600, 900) for box in boxes]
#     normalizes_boxes = [[int(box[0]), int(box[1]), int(box[2]), int(box[3] )] for box in normalizes_boxes]

#     print(normalizes_boxes)
#     # draw predictions over the image
#     draw = ImageDraw.Draw(image)
#     font = ImageFont.load_default()
#     for prediction, box in zip(predictions, normalizes_boxes):
#         if prediction == "O":
#             continue
#         draw.rectangle(box, outline="black")
#         draw.rectangle(box, outline=id2label[prediction])
#         draw.text((box[0] + 10, box[1] - 10), text=prediction, fill=id2label[prediction], font=font)
#     return image


# def run_inference(image, model=model, processor=processor, output_image=True):
#     encoding = processor(image, return_tensors="pt")
#     del encoding["pixel_values"]

#     outputs = model(**encoding)
#     predictions = outputs.logits.argmax(-1).squeeze().tolist()
#     print('pred',predictions)
#     labels = [model.config.id2label[prediction] for prediction in predictions]
#     # if output_image:
#     #     return draw_boxes(image, encoding["bbox"][0], labels)
#     # else:
#     #     return labels
#         # Get label predictions using id2label
#     label_predictions = [id2label[pred] for pred in predictions]
#     # Draw bounding boxes on the image
#     return draw_boxes(image, encoding["bbox"][0].tolist(), labels)

# image_path ="resized_images/20231217_190133_merged-13.pdf_1.png" #"resized_images/FT-230494.pdf_1.png"
# image = Image.open(image_path).convert("RGB")
# # run_inference(image)

# annotated_image = run_inference(image)
# annotated_image.show()


In [ ]:
from transformers import LayoutLMv3ForTokenClassification, LayoutLMv3Processor
from PIL import Image, ImageDraw, ImageFont
import torch
import time  # Import time module for timing

# Initialize model and processor
model = LayoutLMv3ForTokenClassification.from_pretrained("test_trainer/checkpoint-2000-focalloss")
processor = LayoutLMv3Processor.from_pretrained("microsoft/layoutlmv3-base", apply_ocr=True)


def unnormalize_box(bbox, width, height):
    return [
        width * (bbox[0] / 1000),
        height * (bbox[1] / 1000),
        width * (bbox[2] / 1000),
        height * (bbox[3] / 1000),
    ]

# Label mappings
label2id = {
    'O': 0,
    'B-title': 1,
    'I-title': 2,
    'B-date': 3,
    'I-date': 4,
    'B-ieee': 5,
    'I-ieee': 6,
    'B-total': 7,
    'I-total': 8,
    'B-totalValue': 9,
    'I-totalValue': 10
}
id2label = {id: label for label, id in label2id.items()}

def draw_boxes(image, boxes, predictions):
    width, height = image.size
    normalizes_boxes = [unnormalize_box(box, 600, 900) for box in boxes]
    normalizes_boxes = [[int(box[0]), int(box[1]), int(box[2]), int(box[3] )] for box in normalizes_boxes]

    draw = ImageDraw.Draw(image)
    font = ImageFont.load_default()
    for prediction, box in zip(predictions, normalizes_boxes):
        if prediction == "O":
            continue
        draw.rectangle(box, outline="black")
        draw.text((box[0] + 10, box[1] - 10), text=prediction, fill="red", font=font)
    return image

def run_inference(image, model=model, processor=processor, output_image=True):
    # Start timing
    start_time = time.time()
    # Prepare image and run model
    encoding = processor(image, return_tensors="pt")
    outputs = model(**encoding)
    predictions = outputs.logits.argmax(-1).squeeze().tolist()
    elapsed_time = time.time() - start_time
    print(f"Inference time: {elapsed_time:.3f} seconds")

    labels = [model.config.id2label[prediction] for prediction in predictions]

    # End timing


    # Optionally draw bounding boxes and display the image
    if output_image:
        annotated_image = draw_boxes(image, encoding["bbox"][0].tolist(), labels)
        annotated_image.show()

    return labels

image_path = "devis_events-11.pdf_1.png"
image = Image.open(image_path).convert("RGB")

# Run inference and display the annotated image
annotated_image = run_inference(image)


OSError: Incorrect path_or_model_id: '/content/drive/MyDrive/PFA Work - ML Invoice Validation/Advancements/Mariem/models/checkpoint-2000-focalloss-azure'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [ ]:

image_path = "Facture 3 .pdf_1.png"
image = Image.open(image_path).convert("RGB")
# Run inference and display the annotated image

annotated_image = run_inference(image)

Inference time: 1.950 seconds
Inference time: 54.163 seconds


In [ ]:
# !conda install imgaug

In [ ]:
# import imgaug as ia
# ia.seed(1)
# # imgaug uses matplotlib backend for displaying images
# %matplotlib inline
# from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
# from imgaug import augmenters as iaa
# # imageio library will be used for image input/output
# import imageio
# import pandas as pd
# import numpy as np
# import re
# import os
# import glob
# # this library is needed to read XML files for converting it into CSV
# import xml.etree.ElementTree as ET
# import shutil

In [ ]:
# import albumentations as A
# from albumentations.pytorch.transforms import ToTensorV2
# from PIL import Image
# import numpy as np
# import cv2
# import matplotlib.pyplot as plt
# import matplotlib.patches as patches

# def get_transform():
#     return A.Compose([
#         A.Blur(blur_limit=3, p=0.5),
#         A.MedianBlur(blur_limit=3, p=0.5),
#         A.ColorJitter(p=0.5),
#         A.Rotate(limit=10, p=1.0)  # Increased probability for testing
#     ], bbox_params=A.BboxParams(
#         format='pascal_voc',
#         label_fields=['labels']
#     ))


# # Load an image
# image_path = "resized_images/FT-230494.pdf_1.png"
# image = Image.open(image_path).convert("RGB")
# image_np = np.array(image)

# # Define dummy bounding boxes and labels
# # Format: [x_min, y_min, x_max, y_max]
# boxes = [
#         [
#             118.0,
#             35.0,
#             187.0,
#             53.00000000000001
#         ],
#         [
#             193.0,
#             34.0,
#             260.0,
#             53.00000000000001
#         ],
#         [
#             518.0,
#             29.0,
#             567.0,
#             58.0
#         ],
#         [
#             448.0,
#             30.0,
#             513.0,
#             59.00000000000001
#         ],
#         [
#             62.0,
#             65.0,
#             159.0,
#             85.0
#         ],
#         [
#             163.0,
#             66.0,
#             180.99999999999997,
#             83.0
#         ],
#         [
#             185.0,
#             66.0,
#             255.00000000000003,
#             83.0
#         ],
#         [
#             258.0,
#             66.0,
#             279.0,
#             83.0
#         ],
#         [
#             285.0,
#             66.0,
#             312.0,
#             84.0
#         ],
#         [
#             511.0,
#             58.0,
#             570.0,
#             80.0
#         ],
#         [
#             486.00000000000006,
#             58.0,
#             507.00000000000006,
#             81.0
#         ],
#         [
#             447.0,
#             58.0,
#             482.0,
#             81.0
#         ],
#         [
#             126.0,
#             91.0,
#             153.0,
#             104.0
#         ],
#         [
#             156.0,
#             91.0,
#             160.0,
#             104.0
#         ],
#         [
#             162.0,
#             91.0,
#             195.0,
#             104.0
#         ],
#         [
#             198.0,
#             91.0,
#             231.0,
#             103.0
#         ],
#         [
#             234.0,
#             91.0,
#             251.0,
#             103.0
#         ],
#         [
#             442.0,
#             88.0,
#             569.0,
#             106.00000000000001
#         ],
#         [
#             90.0,
#             107.0,
#             117.0,
#             120.0
#         ],
#         [
#             119.0,
#             107.0,
#             145.0,
#             120.0
#         ],
#         [
#             148.0,
#             107.0,
#             182.0,
#             120.0
#         ],
#         [
#             184.0,
#             108.0,
#             203.0,
#             118.99999999999999
#         ],
#         [
#             205.0,
#             108.0,
#             210.00000000000003,
#             118.99999999999999
#         ],
#         [
#             212.0,
#             108.0,
#             261.0,
#             120.0
#         ],
#         [
#             263.0,
#             107.0,
#             287.0,
#             120.0
#         ],
#         [
#             485.0,
#             101.0,
#             525.0,
#             117.0
#         ],
#         [
#             114.0,
#             122.99999999999999,
#             129.0,
#             136.0
#         ],
#         [
#             131.0,
#             122.99999999999999,
#             135.0,
#             136.0
#         ],
#         [
#             137.0,
#             122.99999999999999,
#             149.0,
#             136.0
#         ],
#         [
#             151.0,
#             122.99999999999999,
#             168.99999999999997,
#             136.0
#         ],
#         [
#             170.99999999999997,
#             122.99999999999999,
#             216.0,
#             136.0
#         ],
#         [
#             218.0,
#             124.0,
#             241.0,
#             136.0
#         ],
#         [
#             243.00000000000003,
#             124.0,
#             249.00000000000003,
#             136.0
#         ],
#         [
#             251.0,
#             124.0,
#             264.0,
#             136.0
#         ],
#         [
#             446.0,
#             122.00000000000001,
#             510.0,
#             137.0
#         ],
#         [
#             514.0,
#             122.00000000000001,
#             534.0,
#             137.0
#         ],
#         [
#             540.0,
#             122.00000000000001,
#             563.0,
#             137.0
#         ],
#         [
#             119.0,
#             141.0,
#             147.0,
#             153.0
#         ],
#         [
#             149.0,
#             141.0,
#             154.0,
#             154.00000000000003
#         ],
#         [
#             156.0,
#             141.0,
#             255.00000000000003,
#             154.00000000000003
#         ],
#         [
#             290.0,
#             170.0,
#             311.0,
#             182.0
#         ],
#         [
#             312.99999999999994,
#             170.0,
#             351.0,
#             182.0
#         ],
#         [
#             353.0,
#             170.0,
#             392.0,
#             182.0
#         ],
#         [
#             28.0,
#             189.0,
#             121.0,
#             210.99999999999997
#         ],
#         [
#             290.0,
#             185.0,
#             323.0,
#             196.0
#         ],
#         [
#             325.0,
#             185.0,
#             329.0,
#             196.0
#         ],
#         [
#             331.0,
#             185.0,
#             358.99999999999994,
#             197.0
#         ],
#         [
#             290.0,
#             199.0,
#             302.0,
#             212.00000000000003
#         ],
#         [
#             304.0,
#             200.0,
#             308.00000000000006,
#             212.0
#         ],
#         [
#             310.00000000000006,
#             200.0,
#             349.00000000000006,
#             212.0
#         ],
#         [
#             351.0,
#             200.0,
#             378.0,
#             212.0
#         ],
#         [
#             380.0,
#             200.0,
#             397.0,
#             212.0
#         ],
#         [
#             138.0,
#             232.0,
#             150.0,
#             243.00000000000003
#         ],
#         [
#             152.0,
#             232.0,
#             188.0,
#             243.99999999999997
#         ],
#         [
#             424.0,
#             232.0,
#             444.0,
#             242.0
#         ],
#         [
#             139.0,
#             248.0,
#             188.0,
#             261.0
#         ],
#         [
#             407.0,
#             249.0,
#             463.00000000000006,
#             262.0
#         ],
#         [
#             31.0,
#             286.0,
#             88.0,
#             299.0
#         ],
#         [
#             346.0,
#             286.0,
#             361.99999999999994,
#             297.0
#         ],
#         [
#             406.0,
#             286.0,
#             420.0,
#             296.0
#         ],
#         [
#             424.0,
#             286.0,
#             438.0,
#             296.0
#         ],
#         [
#             465.0,
#             286.0,
#             483.00000000000006,
#             296.0
#         ],
#         [
#             511.0,
#             285.0,
#             550.0,
#             296.99999999999994
#         ],
#         [
#             553.0,
#             286.0,
#             567.0,
#             297.0
#         ],
#         [
#             31.0,
#             305.0,
#             68.0,
#             319.0
#         ],
#         [
#             70.0,
#             305.0,
#             97.00000000000001,
#             319.0
#         ],
#         [
#             99.0,
#             305.0,
#             140.0,
#             319.0
#         ],
#         [
#             142.0,
#             305.0,
#             165.0,
#             319.0
#         ],
#         [
#             168.00000000000003,
#             305.0,
#             195.0,
#             319.0
#         ],
#         [
#             198.0,
#             306.0,
#             217.00000000000003,
#             319.0
#         ],
#         [
#             219.0,
#             306.0,
#             238.0,
#             319.0
#         ],
#         [
#             241.0,
#             306.0,
#             263.0,
#             319.0
#         ],
#         [
#             352.0,
#             307.0,
#             356.0,
#             315.0
#         ],
#         [
#             380.0,
#             305.0,
#             416.0,
#             317.0
#         ],
#         [
#             418.0,
#             306.0,
#             436.99999999999994,
#             317.00000000000006
#         ],
#         [
#             473.0,
#             306.0,
#             483.00000000000006,
#             315.00000000000006
#         ],
#         [
#             509.00000000000006,
#             306.0,
#             545.0000000000001,
#             317.00000000000006
#         ],
#         [
#             547.0,
#             306.0,
#             566.0,
#             317.00000000000006
#         ],
#         [
#             58.0,
#             689.0,
#             80.0,
#             699.0
#         ],
#         [
#             82.0,
#             689.0,
#             96.99999999999999,
#             699.0
#         ],
#         [
#             148.0,
#             689.0,
#             153.0,
#             700.0
#         ],
#         [
#             155.00000000000003,
#             689.0,
#             173.0,
#             700.0
#         ],
#         [
#             216.0,
#             688.0,
#             257.0,
#             700.0
#         ],
#         [
#             259.0,
#             689.0,
#             277.0,
#             700.0
#         ],
#         [
#             69.0,
#             705.0,
#             103.0,
#             715.0
#         ],
#         [
#             107.0,
#             705.0,
#             124.0,
#             715.0
#         ],
#         [
#             161.0,
#             704.0,
#             171.99999999999997,
#             715.0000000000001
#         ],
#         [
#             178.0,
#             704.0,
#             184.00000000000003,
#             715.0000000000001
#         ],
#         [
#             229.0,
#             704.0,
#             259.0,
#             716.0
#         ],
#         [
#             261.0,
#             705.0,
#             280.0,
#             716.0
#         ],
#         [
#             321.0,
#             695.0,
#             344.0,
#             706.0
#         ],
#         [
#             345.0,
#             695.0,
#             358.99999999999994,
#             706.0
#         ],
#         [
#             507.0,
#             695.0,
#             542.0,
#             706.0
#         ],
#         [
#             546.0,
#             695.0,
#             564.0,
#             706.0
#         ],
#         [
#             27.0,
#             719.0,
#             50.0,
#             732.0
#         ],
#         [
#             53.0,
#             720.0,
#             63.0,
#             732.0
#         ],
#         [
#             65.0,
#             720.0,
#             107.0,
#             732.0
#         ],
#         [
#             109.0,
#             720.0,
#             114.0,
#             732.0
#         ],
#         [
#             321.0,
#             708.0,
#             340.0,
#             719.0
#         ],
#         [
#             512.0,
#             709.0,
#             542.0000000000001,
#             721.0
#         ],
#         [
#             545.0,
#             709.0,
#             564.0,
#             720.0
#         ],
#         [
#             321.0,
#             723.0,
#             344.0,
#             734.0
#         ],
#         [
#             345.0,
#             723.0,
#             355.99999999999994,
#             734.0
#         ],
#         [
#             357.0,
#             723.0,
#             387.99999999999994,
#             734.0
#         ],
#         [
#             518.0,
#             723.0,
#             542.0,
#             734.0
#         ],
#         [
#             544.0,
#             723.0,
#             563.0,
#             734.0
#         ],
#         [
#             322.0,
#             740.0,
#             352.99999999999994,
#             751.0
#         ],
#         [
#             356.0,
#             740.0,
#             374.00000000000006,
#             751.0
#         ],
#         [
#             503.0,
#             739.0,
#             542.0,
#             752.0
#         ],
#         [
#             544.0,
#             739.0,
#             564.0,
#             752.0
#         ],
#         [
#             27.0,
#             760.0,
#             50.0,
#             773.0
#         ],
#         [
#             53.0,
#             760.0,
#             60.0,
#             773.0
#         ],
#         [
#             62.0,
#             760.0,
#             96.0,
#             773.0
#         ],
#         [
#             98.0,
#             760.0,
#             126.0,
#             773.0
#         ],
#         [
#             128.0,
#             760.0,
#             132.0,
#             773.0
#         ],
#         [
#             134.0,
#             760.0,
#             142.0,
#             773.0
#         ],
#         [
#             144.0,
#             760.0,
#             172.0,
#             773.0
#         ],
#         [
#             174.0,
#             760.0,
#             183.0,
#             773.0
#         ],
#         [
#             185.0,
#             760.0,
#             188.0,
#             773.0
#         ],
#         [
#             190.0,
#             760.0,
#             213.0,
#             773.0
#         ],
#         [
#             215.0,
#             760.0,
#             236.0,
#             773.0
#         ],
#         [
#             238.0,
#             760.0,
#             279.0,
#             773.0
#         ],
#         [
#             281.0,
#             760.0,
#             296.0,
#             772.0
#         ],
#         [
#             298.0,
#             760.0,
#             331.0,
#             772.0
#         ],
#         [
#             103.0,
#             819.0,
#             119.99999999999999,
#             833.0
#         ],
#         [
#             122.99999999999999,
#             819.0,
#             134.0,
#             833.0
#         ],
#         [
#             136.0,
#             819.0,
#             170.99999999999997,
#             833.0
#         ],
#         [
#             174.0,
#             819.0,
#             179.0,
#             833.0
#         ],
#         [
#             181.00000000000003,
#             819.0,
#             190.00000000000003,
#             833.0
#         ],
#         [
#             193.0,
#             819.0,
#             223.0,
#             833.0
#         ],
#         [
#             225.0,
#             819.0,
#             265.0,
#             833.0
#         ],
#         [
#             268.0,
#             819.0,
#             281.0,
#             833.0
#         ],
#         [
#             283.0,
#             819.0,
#             320.0,
#             833.0
#         ],
#         [
#             322.0,
#             819.0,
#             338.99999999999994,
#             833.0
#         ],
#         [
#             341.0,
#             819.0,
#             362.00000000000006,
#             833.0
#         ],
#         [
#             364.0,
#             819.0,
#             375.0,
#             833.0
#         ],
#         [
#             378.0,
#             819.0,
#             398.0,
#             834.0000000000001
#         ],
#         [
#             401.0,
#             819.0,
#             447.0,
#             834.0000000000001
#         ],
#         [
#             450.0,
#             819.0,
#             496.0,
#             834.0000000000001
#         ],
#         [
#             76.0,
#             835.0,
#             93.0,
#             849.0
#         ],
#         [
#             95.0,
#             835.0,
#             100.0,
#             849.0
#         ],
#         [
#             102.00000000000001,
#             835.0,
#             107.0,
#             849.0
#         ],
#         [
#             109.0,
#             835.0,
#             114.0,
#             849.0
#         ],
#         [
#             116.0,
#             835.0,
#             119.99999999999999,
#             849.0
#         ],
#         [
#             122.0,
#             835.0,
#             140.0,
#             849.0
#         ],
#         [
#             142.0,
#             835.0,
#             203.99999999999997,
#             849.0
#         ],
#         [
#             206.99999999999997,
#             835.0,
#             236.0,
#             849.0
#         ],
#         [
#             238.0,
#             835.0,
#             260.0,
#             849.0
#         ],
#         [
#             262.0,
#             835.0,
#             267.0,
#             849.0
#         ],
#         [
#             269.0,
#             835.0,
#             292.0,
#             849.0
#         ],
#         [
#             294.0,
#             835.0,
#             299.0,
#             849.0
#         ],
#         [
#             302.0,
#             835.0,
#             314.0,
#             849.0
#         ],
#         [
#             315.99999999999994,
#             835.0,
#             320.99999999999994,
#             849.0
#         ],
#         [
#             323.0,
#             835.0,
#             335.0,
#             849.0
#         ],
#         [
#             337.0,
#             835.0,
#             354.0,
#             849.0
#         ],
#         [
#             356.0,
#             835.0,
#             375.0,
#             849.0
#         ],
#         [
#             377.0,
#             835.0,
#             381.0,
#             849.0
#         ],
#         [
#             383.0,
#             835.0,
#             401.99999999999994,
#             849.0
#         ],
#         [
#             406.0,
#             835.0,
#             410.0,
#             849.0
#         ],
#         [
#             412.0,
#             835.0,
#             424.0,
#             849.0
#         ],
#         [
#             426.0,
#             835.0,
#             441.99999999999994,
#             849.0
#         ],
#         [
#             445.0,
#             835.0,
#             461.0,
#             849.0
#         ],
#         [
#             462.99999999999994,
#             835.0,
#             469.0,
#             849.0
#         ],
#         [
#             471.0,
#             835.0,
#             482.0,
#             848.0000000000001
#         ],
#         [
#             485.0,
#             835.0,
#             501.0,
#             848.0000000000001
#         ],
#         [
#             503.0,
#             835.0,
#             522.0,
#             848.0000000000001
#         ],
#         [
#             117.0,
#             852.0,
#             134.0,
#             866.0
#         ],
#         [
#             136.0,
#             852.0,
#             141.0,
#             866.0
#         ],
#         [
#             143.0,
#             852.0,
#             148.0,
#             866.0
#         ],
#         [
#             150.0,
#             852.0,
#             167.0,
#             866.0
#         ],
#         [
#             169.0,
#             852.0,
#             221.0,
#             866.0
#         ],
#         [
#             223.0,
#             852.0,
#             228.0,
#             866.0
#         ],
#         [
#             230.00000000000003,
#             852.0,
#             253.0,
#             866.0
#         ],
#         [
#             255.0,
#             852.0,
#             260.0,
#             866.0
#         ],
#         [
#             262.0,
#             852.0,
#             275.0,
#             866.0
#         ],
#         [
#             277.0,
#             852.0,
#             282.0,
#             866.0
#         ],
#         [
#             284.0,
#             852.0,
#             295.0,
#             866.0
#         ],
#         [
#             297.0,
#             852.0,
#             314.0,
#             866.0
#         ],
#         [
#             315.99999999999994,
#             851.0,
#             332.99999999999994,
#             866.0000000000001
#         ],
#         [
#             335.0,
#             851.0,
#             341.0,
#             866.0000000000001
#         ],
#         [
#             343.0,
#             851.0,
#             361.99999999999994,
#             866.0000000000001
#         ],
#         [
#             364.99999999999994,
#             851.0,
#             369.0,
#             866.0000000000001
#         ],
#         [
#             371.0,
#             851.0,
#             383.0,
#             866.0000000000001
#         ],
#         [
#             385.00000000000006,
#             851.0,
#             402.0,
#             866.0000000000001
#         ],
#         [
#             404.0,
#             851.0,
#             421.0,
#             866.0000000000001
#         ],
#         [
#             423.0,
#             851.0,
#             427.99999999999994,
#             866.0000000000001
#         ],
#         [
#             430.0,
#             851.0,
#             442.0,
#             866.0000000000001
#         ],
#         [
#             444.0,
#             851.0,
#             461.0,
#             866.0000000000001
#         ],
#         [
#             462.0,
#             851.0,
#             483.00000000000006,
#             866.0000000000001
#         ],
#         [
#             555.0,
#             870.0,
#             571.0,
#             882.0
#         ]
#     ]
# labels = [
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "B-ieee",
#         "I-ieee",
#         "I-ieee",
#         "B-title",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "B-date",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "B-total",
#         "I-total",
#         "B-totalValue",
#         "I-totalValue",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O",
#         "O"
#     ]
# # Apply transformation
# transform = get_transform()
# transformed = transform(image=image_np, bboxes=boxes, labels=labels)
# transformed_image = transformed['image']
# transformed_boxes = transformed['bboxes']

# def visualize(image, boxes, title="Transformed Image"):
#     fig, ax = plt.subplots(1)
#     ax.imshow(image)
#     for box in boxes:
#         rect = patches.Rectangle((box[0], box[1]), box[2] - box[0], box[3] - box[1], linewidth=1, edgecolor='r', facecolor='none')
#         ax.add_patch(rect)
#     plt.title(title)
#     plt.show()

# # Visualize original and transformed images
# visualize(image_np, boxes, title="Original Image")
# visualize(transformed_image, transformed_boxes, title="Transformed Image")
